## Iowa Liquor Sales

### Import Package

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from datetime import time
from datetime import datetime
import datetime

In this task we will explore the data of Iowa Liquor Sales. Since the data is very large we will to some sample from the data using a SQL query in the Google BigQuery. Before we discuss the query we need to explain how we going to take the sample.

Firstly, we took the data between 01 January 2020 to 29 January 2021 because the data is much more updated since the maximum date is on 29 January 2021. We will be looking at the number of Transaction in every store between that date. The query is

`SELECT * 
FROM bigquery-public-data.iowa_liquor_sales.sales 
where (date BETWEEN '2020-01-01' AND '2021-01-29')`

the result will be below.

In [ ]:
df = pd.read_csv("full_data.csv")

In [ ]:
sales_total = df.groupby('store_number').sum()[["sale_dollars"]]

In [ ]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=sales_total["sale_dollars"],
                     name="sale dollars"))

fig.update_layout(title_text="Box Plot Every total sales in all store between 01 January 2020 to 29 January 2021")

fig.show()

We can see from the boxplot there are some store that have high sales we need to compare how can this sales grow more compare to the minimum sales.

### Exploration Dataset

Firstly, we will look at the dataset if there is a null value.

In [ ]:
# print(df.isnull().sum())

As you can see that there are some Null value the store location. This might be an unknown transaction or fraud transactions since the are no location. so we need to filter the data and fill some of the missing value

In [ ]:
df['store_location'] = df['store_location'].fillna('NA')
df['category'] = df['category'].fillna(0)
df['vendor_number'] = df['vendor_number'].fillna(0)
df['category_name'] = df['category_name'].fillna('other').str.upper()
df['vendor_name'] = df['vendor_name'].fillna('other').str.upper()
df['county'] = df['county'].str.upper()
df['city'] = df['city'].str.upper()
df['item_description'] = df['item_description'].str.upper()


df = df[(df['store_location'] != "NA")]
df.drop_duplicates()
df = df.reset_index(drop=True)

In [ ]:
full_df_cat = df.groupby(['category_name','pack','date'])['sale_dollars'].sum().groupby(
            ['category_name','pack']).max().sort_values().groupby(
            ['category_name']).sum().sort_values(ascending=False)

top_count10_cat = pd.DataFrame(full_df_cat.head(10))

In [ ]:
fig_reg = px.bar(top_count10_cat,x=top_count10_cat.index, y='sale_dollars',color='sale_dollars')
fig_reg.update_layout(
    title="Top 10 Sales of liquor per category 01 January 2020 to 29 January 2021",
    xaxis_title="Category Name",
    yaxis_title="Sales in dollars",
    )
fig_reg.show()

In [ ]:
full_df_city = df.groupby(['city','pack','date'])['sale_dollars'].sum().groupby(
            ['city','pack']).max().sort_values().groupby(
            ['city']).sum().sort_values(ascending=False)

top_count10_city = pd.DataFrame(full_df_city.head(10))

In [ ]:
fig_reg = px.bar(top_count10_city,x=top_count10_city.index, y='sale_dollars',color='sale_dollars')
fig_reg.update_layout(
    title="Top 10 Sales of liquor per city 01 January 2020 to 29 January 2021",
    xaxis_title="City Name",
    yaxis_title="Sales in dollars",
    )
fig_reg.show()

In [ ]:
daily_sales = df.groupby('date', as_index=False)['sale_dollars'].sum()

daily_sales_sc = go.Scatter(x=daily_sales['date'], y=daily_sales['sale_dollars'])
layout = go.Layout(title='Daily sales 01 January 2020 to 29 January 2021', xaxis=dict(title='Date'), yaxis=dict(title='Sales'))
fig = go.Figure(data=[daily_sales_sc], layout=layout)
fig.show()

In [ ]:
full_df_county = df.groupby(['county','pack','date'])['sale_dollars'].sum().groupby(
            ['county','pack']).max().sort_values().groupby(
            ['county']).sum().sort_values(ascending=False)

top_count10_county = pd.DataFrame(full_df_county.head(10))

fig_reg = px.bar(top_count10_county,x=top_count10_county.index, y='sale_dollars',color='sale_dollars')
fig_reg.update_layout(
    title="Sales of liquor per category",
    xaxis_title="county Name",
    yaxis_title="Sales in dollars",
    )

fig_reg.show()

### Covid-19 Situation 

Some people want to grab a during this pandemic situation we want to see the correlation of the covid-19 case and the daily sales, we extract some data from {https://open-iowa.opendata.arcgis.com/datasets/6a84756c2e444a87828bb7ce699fdac6_0/data?selectedAttribute=Country} and we combine the data and create a line chart.

In [ ]:
covid_iowa = pd.read_csv("Individuals Positive.csv").rename(columns = {"Date": "date"})

In [ ]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(
    go.Scatter(x=daily_sales["date"], y=daily_sales['sale_dollars'], name="Liquor Sales"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=covid_iowa["date"], y=covid_iowa["Individuals Positive - 14 Day Rolling Total"], name="Covid Case"),
    row=2, col=1
)

fig.update_layout(height=600, width=800, title_text="Iowa Liquor Sales Total and Covid Case Total")
fig.show()

In [ ]:
fips = pd.read_excel("cofipsia.xlsx").rename(columns = {"County Name":"county"})
fips['county'] = fips['county'].str.replace(" County", "").str.upper()
df_new = df.merge(fips, on = "county")

In [ ]:
full_df_county_fips = df_new.groupby(['county', 'FIPS Code','pack','date'])['sale_dollars'].sum().groupby(
            ['county', 'FIPS Code','pack']).max().sort_values().groupby(
            ['county', 'FIPS Code']).sum().sort_values(ascending=False)

fips_sales = pd.DataFrame(full_df_county_fips).reset_index()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
    
fig = px.choropleth(fips_sales, geojson=counties, locations='FIPS Code', color='sale_dollars',
                           color_continuous_scale="Viridis",
                           scope="usa",
                           hover_name="county",
                           labels={'unemp':'sale_dollars'}
                          )

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text = 'Total Sales Iowa')
fig.show()

In [ ]:
covid_iowa_state = pd.read_csv("summary.csv").rename(columns = {'County':'county'})
covid_iowa_state["county"] = covid_iowa_state["county"].str.upper()

covid_fips = covid_iowa_state.merge(fips, on = "county")

In [ ]:
fig = px.choropleth(covid_fips, geojson=counties, locations='FIPS Code', color='Individuals Positive',
                           color_continuous_scale="Viridis",
                           scope="usa",
                           hover_name="county",
                           labels={'unemp':'Individuals Positive'}
                          )

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text = 'Covid Case Iowa')
fig.show()

In [ ]:
df_plot = df_new.groupby(['category_name','pack','date'])['sale_dollars'].sum().groupby(
            ['category_name','pack']).max().sort_values().groupby(
            ['category_name']).sum().sort_values(ascending=False)
top_count1_sale = pd.DataFrame(df_plot)

df_plot = df_new.groupby(['category_name','pack','date'])['volume_sold_liters'].sum().groupby(
            ['category_name','pack']).max().sort_values().groupby(
            ['category_name']).sum().sort_values(ascending=False)
top_count2_liter = pd.DataFrame(df_plot)

In [ ]:
top_count = pd.concat([top_count1_sale , top_count2_liter],axis=1)
top_count = top_count.sort_values(['sale_dollars'],ascending=False)[:10]


In [ ]:
fig = go.Figure(data=[
    go.Bar(name='sale in dollars',x=top_count.index, y=top_count['sale_dollars']),
    go.Bar(name='Volume in litres',x=top_count.index, y=top_count['volume_sold_liters'])
])
# Change the bar mode
fig.update_layout(barmode='group',title="Sales of liquors with category and the volume of liquor sold.",
    xaxis_title=" category",
    yaxis_title="Sale and the amount of liquor sold in litre.",)
fig.show()

In [ ]:
df_plot = df_new.groupby(['vendor_name','pack','date'])['volume_sold_liters'].sum().groupby(
            ['vendor_name','pack']).max().sort_values().groupby(
            ['vendor_name']).sum().sort_values(ascending=False)

top_count1 = pd.DataFrame(df_plot.head(20))

fig_reg = px.bar(top_count1,x=top_count1.index, y='volume_sold_liters',color='volume_sold_liters')
fig_reg.update_layout(
    title="Top 20 Volume sold by vendor name",
    xaxis_title=" Vendor Name",
    yaxis_title="Liquor sold in Litres",
    )
fig_reg.show()

### RFM Analysis

In this section we will do a store segmentation to see the behavior of every store. we will categorize in this part.

Let’s delve into few interesting segments:

- **Champions** are the best buyers, who have recently bought, most regularly, and are big spenders. And compensate these clients. For emerging products, they may become early adopters and help market the brand.

- **Potential Loyalists** are average frequency of your latest consumers and who paid a decent amount. Give or suggest similar items to subscription or loyalty groups to upsell them and make them become the Loyalists or Champions.

- **New Customers** are The clients who have a high RFM average score but are not regular shoppers. Start building partnerships with these consumers to maximize their visits by offering onboarding assistance and promotional deals.

- **At Risk Customers** are your clients who ordered regularly and paid significant sums, but lately did not buy. To promote another order, give them customized reactivation campaigns to reconnect, and deliver renewals and beneficial goods.

- **Can’t Lose Them** are customers who, very frequently, used to visit and order, but have not visited lately. To figure out what went wrong and stop losing them to a rival, get them back with appropriate incentives and run surveys.

We will use this analysis to check the potential store so that we can develop and give them some suggestion about the sales.

In [ ]:
Frequency = df_new.groupby("store_number").count()[["invoice_and_item_number"]]

In [ ]:
MonetaryValue = df_new.groupby("store_number").sum()[["sale_dollars"]]

In [ ]:
df_new['date'] = pd.to_datetime(df_new['date'])

In [ ]:
date_df = df_new.groupby(["store_number", 'date']).count().reset_index()[["store_number", 'date']]
list_cust = list(set(df_new["store_number"]))
last_purch = []
for i in list_cust:
    c = datetime.datetime(2021, 1, 30) - date_df[date_df["store_number"] == i]["date"][-1:]
    c = c/np.timedelta64(1, 'D')
    last_purch.append(int(c))

In [ ]:
Recency = pd.DataFrame({"store_number":list_cust, "last_transaction_day":  last_purch}).set_index("store_number")

In [ ]:
df_rfm = pd.concat([Recency, Frequency, MonetaryValue], axis=1)

In [ ]:
# Create labels for Recency and Frequency
r_labels = range(4, 0, -1); f_labels = range(1, 5)

# Assign these labels to 4 equal percentile groups 
r_groups = pd.qcut(df_rfm['last_transaction_day'], q=4, labels=r_labels)
# Assign these labels to 4 equal percentile groups 
f_groups = pd.qcut(df_rfm['invoice_and_item_number'], q=4, labels=f_labels)

In [ ]:
df_rfm = df_rfm.assign(R = r_groups.values, F = f_groups.values)
# Create labels for MonetaryValue
m_labels = range(1, 5)
# Assign these labels to three equal percentile groups 
m_groups = pd.qcut(df_rfm['sale_dollars'], q=4, labels=m_labels)
# Create new column M
df_rfm = df_rfm.assign(M = m_groups.values)

In [ ]:
df_rfm['RFM_Score'] = df_rfm[['R','F','M']].sum(axis=1)

In [ ]:
def rfm_level(df):
    if df['RFM_Score'] >= 9:
        return 'Can\'t Loose Them'
    elif ((df['RFM_Score'] >= 8) and (df['RFM_Score'] < 9)):
        return 'Champions'
    elif ((df['RFM_Score'] >= 7) and (df['RFM_Score'] < 8)):
        return 'Loyal'
    elif ((df['RFM_Score'] >= 6) and (df['RFM_Score'] < 7)):
        return 'Potential'
    elif ((df['RFM_Score'] >= 5) and (df['RFM_Score'] < 6)):
        return 'Promising'
    elif ((df['RFM_Score'] >= 4) and (df['RFM_Score'] < 5)):
        return 'Needs Attention'
    else:
        return 'Require Activation'

# Create a new variable RFM_Level
df_rfm['RFM_Level'] = df_rfm.apply(rfm_level, axis=1)
# Print the header with top 5 rows to the console

In [ ]:
rfm_level = pd.DataFrame(df_rfm[["RFM_Level"]].value_counts()).reset_index()

fig_reg = px.bar(rfm_level,x="RFM_Level", y = 0, color="RFM_Level")

fig_reg.update_layout(
    title="Number of Store based on level",
    xaxis_title="Level",
    yaxis_title="Count total",
    )
fig_reg.show()

In [ ]:
fig = px.scatter(df_rfm, x="invoice_and_item_number", y="last_transaction_day", color="RFM_Level")
fig.update_layout(
    title="Recency vs Frequency",
    xaxis_title="Frequency",
    yaxis_title="Recency",
    )
fig.show()

fig = px.scatter(df_rfm, x="sale_dollars", y="invoice_and_item_number", color="RFM_Level")
fig.update_layout(
    title="MonetaryValue vs Frequency",
    xaxis_title="MonetaryValue",
    yaxis_title="Frequency",
    )
fig.show()

fig = px.scatter(df_rfm, x="last_transaction_day", y="sale_dollars", color="RFM_Level")
fig.update_layout(
    title="Recency vs MonetaryValue",
    xaxis_title="Recency",
    yaxis_title="MonetaryValue",
    )
fig.show()